## Initialization

In [3]:
import matplotlib.pyplot as plt
from mog_model import *
import pickle
%autoreload 2

In [2]:
X_DIM = 2
NUM_DISTRIBUTIONS = 2

MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
CENTERS = np.sqrt(2)  # center of Gaussian
for i in range(NUM_DISTRIBUTIONS):
    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

params = {                          # default parameter values
    'x_dim': X_DIM,
    'num_distributions': NUM_DISTRIBUTIONS,
    'means': MEANS,
    'sigma': 0.05,
    'small_pi': 2E-16,
    'scale': 0.1,
    'num_samples': 200,
    'lr_init': 1e-3,
    'lr_decay_steps': 1000,
    'lr_decay_rate': 0.96,
    'eps': 0.5,
    'temp_init': 20,
    'annealing_steps': 200,
    'annealing_rate': 0.98,
    #'train_trajectory_length': 15,
    #'test_trajectory_length': 2000,
    'num_training_steps': 30000,
    'tunneling_rate_steps': 500,
    'save_steps': 1000,
    'logging_steps': 100
}

## Build / Train model

In [4]:
tf.reset_default_graph()

In [5]:
#%pdb
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
model = GaussianMixtureModel(params, 
                             config=config)
                             #log_dir='../log_mog_tf/run192/')
                             #log_dir='../log_mog_tf/run64/')

Creating directory for new run: ../log_mog_tf/run_236/
Initial parameters written to ../log_mog_tf/run_236/run_info/_init_params.pkl.
Saving parameter values to: ../log_mog_tf/run_236/run_info/
params file written to: ../log_mog_tf/run_236/run_info/parameters.txt
################################################################################
Model parameters:
x_dim: 2

num_distributions: 2

eps: 0.5

scale: 0.1

num_samples: 200

sigma: 0.05

small_pi: 2e-16

lr_init: 0.001

temp_init: 20

annealing_steps: 200

annealing_factor: 0.98

num_training_steps: 30000

tunneling_rate_steps: 500

lr_decay_steps: 1000

lr_decay_rate: 0.96

logging_steps: 100

save_steps: 1000

annealing_rate: 0.98

temp: 20

step_init: 0

log_dir: ../log_mog_tf/run_236/

info_dir: ../log_mog_tf/run_236/run_info/

figs_dir: ../log_mog_tf/run_236/figures/

trajectory_length: 3.0

################################################################################




In [6]:
model.train(params['num_training_steps'])

---------------------------------------------------------------------------------------
    STEP       TEMP      LOSS      ACCEPT RATE      LR        STEP SIZE    TRAJ LEN   
---------------------------------------------------------------------------------------
    0/30000    19.6     -80.88       0.9205        0.001       0.5005          2      
  100/30000    19.6     -190.8        0.921        0.001        0.546          2      
  200/30000   19.21     -187.9       0.9626        0.001       0.5434          2      
  300/30000   19.21     -193.6       0.9426        0.001       0.5427          2      
  400/30000   18.82     -204.3       0.9458        0.001       0.5411          2      
  STEP   TEMP  LOSS  ACCEPT %   ERR     TUNN %    ERR      DIST     ERR    STEP SIZE LENGTH
  500     1    2000     0        0        0        0    1.399e-08 5.774e-10  0.5385     2   
---------------------------------------------------------------------------------------------
  500    18.8 -127.9  0

Saving figure to: ../log_mog_tf/run_236/figures/tr_ar_dist_temps_lowT.pdf
Saving figure to: ../log_mog_tf/run_236/figures/tr_ar_dist_temps_highT.pdf

Tunneling rate increased. Speeding up annealing schedule.
Change in tunneling rate (temp = 1): 0.0
Change in tunneling rate (temp = 15.4): 0.11740058307893153
Annealing steps: 100 --> 50
Temperature: 15.4 --> 15.4

---------------------------------------------------------------------------------------
    STEP       TEMP      LOSS      ACCEPT RATE      LR        STEP SIZE    TRAJ LEN   
---------------------------------------------------------------------------------------
 2000/30000   15.07     -245.4        0.948      0.0009216     0.5279          2      
 2100/30000   14.48     -243.4       0.9327      0.0009216     0.5289          2      
 2200/30000    13.9     -238.6       0.9398      0.0009216     0.5244          2      
 2300/30000   13.35     -254.1        0.939      0.0009216     0.5199          2      
 2400/30000   12.82     

Saving figure to: ../log_mog_tf/run_236/figures/tr_ar_dist_temps_highT.pdf

Tunneling rate increased. Speeding up annealing schedule.
Change in tunneling rate (temp = 1): 0.002893672013318703
Change in tunneling rate (temp = 6.86): -0.0008748687974177161
Annealing steps: 50 --> 0
Temperature: 6.86 --> 6.86

---------------------------------------------------------------------------------------
    STEP       TEMP      LOSS      ACCEPT RATE      LR        STEP SIZE    TRAJ LEN   
---------------------------------------------------------------------------------------


ZeroDivisionError: integer division or modulo by zero

### Less old

In [ ]:
for key in model.tunneling_rates_highT.keys():
    #print(f'Step num: {key[0]}')
    #print(f'Temp: {key[1]}')
    model.steps_arr.append(key[0])
    model.temp_arr.append(key[1])
    #model.temp_arr.append(key[1])

In [ ]:
for idx in range(len(model.steps_arr)):
    model.steps_arr[idx] += 1

In [ ]:
model.steps_arr

In [ ]:
for key, val in model.tunneling_rates.items():
    model.tunneling_rates_avg.append(val[0])
    model.tunneling_rates_err.append(val[1])
for key, val in model.tunneling_rates_highT.items():
    model.tunneling_rates_avg_highT.append(val[0])
    model.tunneling_rates_err_highT.append(val[1])
    
for key, val in model.acceptance_rates.items():
    model.acceptance_rates_avg.append(val[0])
    model.acceptance_rates_err.append(val[1])
for key, val in model.acceptance_rates_highT.items():
    model.acceptance_rates_avg_highT.append(val[0])
    model.acceptance_rates_err_highT.append(val[1])
    
for key, val in model.distances.items():
    model.distances_avg.append(val[0])
    model.distances_err.append(val[1])
for key, val in model.distances_highT.items():
    model.distances_avg_highT.append(val[0])
    model.distances_err_highT.append(val[1])

In [ ]:
model._save_variables()

In [ ]:
model._init_params(params)

In [ ]:
model._load_variables()

In [ ]:
model.steps_arr

In [ ]:
model.tunneling_rates

In [ ]:
import os

In [ ]:
model.__dict__.keys()

In [ ]:
get_vals_as_arr = lambda _dict: np.array(list(_dict.values()))
tr = get_vals_as_arr(model.tunneling_rates)
ar = get_vals_as_arr(model.acceptance_rates)
dr = get_vals_as_arr(model.distances)
#np.array(list(model.tunneling_rates.values()))

In [ ]:
for key in model.attrs_dict.keys():
    in_file = model.info_dir + key + '.npy'
    if os.path.isfile(in_file):
        setattr(model, key, np.load(in_file))
        print(f'Set model.{key} to values read in from: {in_file}')
        #print(model.key == np.load(in_file))

In [ ]:
model.sess.run(tf.global_variables_initializer())

In [ ]:
attrs_dict = model.attrs_dict

In [ ]:
attrs_dict['steps_arr']

In [ ]:
model.train(params['num_training_steps'], plot=True)

## OLD

In [ ]:
#trajectories, loss_arr, px_arr = model.generate_trajectories(num_samples=100, num_steps=100, temperature=1.)

#config = tf.ConfigProto(log_device_placement=True)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#model.build_graph()

In [ ]:
def calc_avg_distance1(trajectories):
    distances_arr = []
    for trajectory in trajectories:
        distance_arr = []
        for idx in range(1, len(trajectory)):
            diff = trajectory[idx] - trajectory[idx-1]
            dist = np.sqrt(np.dot(diff, diff.T))
            distance_arr.append(dist)
        distances_arr.append(sum(distance_arr))
    return np.mean(distances_arr)

def calc_avg_distance2(trajectories):
    dist = lambda d: np.sqrt(np.dot(d, d.T))
    #distances_arr = np.mean([[dist(d) for d in [t[:-1, :] - t[1:, :] for t in trajectories]]
    #for trajectory in trajectories:
        diff = trajectory[:-1, :] - trajectory[1:, :]
        distance = sum([np.sqrt(np.dot(d, d.T)) for d in diff])
        distances_arr.append(distance)
    return np.mean(distances_arr)

In [ ]:
%timeit avg_dist = avg_distance_traveled(trajectories.transpose([1,0,2]))
#print(avg_dist)

%timeit avg_dist1 = calc_avg_distance1(trajectories.transpose([1,0,2]))
#print(avg_dist1)

%timeit avg_dist2 = calc_avg_distance2(trajectories.transpose([1,0,2]))
#print(avg_dist2)

distances = np.array([np.sqrt(np.dot(disp, disp.T)) for disp in displacements])

In [ ]:
model.train(params['num_training_steps'], config=config, plot=True)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)